In [1]:
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [200]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical

In [331]:
from keras.preprocessing.sequence import pad_sequences

In [151]:
data_path='/models'

In [504]:
df = pd.read_pickle('lstm_1.pkl')

In [505]:
len(df)

60289

In [510]:
len(df.loc[df.answers_lstm_len==2])

32629

In [516]:
df2 = df.loc[df.answers_lstm_len==2]

In [517]:
df.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
1,sing dance,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
2,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,83,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
3,houston texas,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
4,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...


In [682]:
ndf = df2.sample(frac=1).reset_index(drop=True)


In [519]:
len(ndf)

32629

In [683]:
ndf = ndf[:5000]

In [684]:
len(ndf)

5000

In [685]:
ndf.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,nato,two controversy campaign test staff affect cam...,72,2,two controversy campaign test staff affect cam...
1,five,john first wife isabel countess gloucester rel...,132,2,john first wife isabel countess gloucester rel...
2,180000300000,half jew live diaspora see population table cu...,103,2,half jew live diaspora see population table cu...
3,priscillian,within six year official criminalization heres...,117,2,within six year official criminalization heres...
4,xiamen,xiamen dialect sometimes know amoy main dialec...,104,2,xiamen dialect sometimes know amoy main dialec...


In [686]:
ndf.text_len.max()

315

In [687]:
def load_data():
    # 
    ndf.reset_index(inplace=True, drop=True)
    
    train_df = ndf.iloc[:round(0.8*len(ndf))]
    valid_df = ndf.iloc[round(0.8*len(ndf)):round(0.9*len(ndf))]
    test_df = ndf.iloc[round(0.9*len(ndf)):]
    

    # build the complete vocabulary, then convert text data to dict of integer-word pairs
    vocabulary = lib.get_total_vocab(ndf, column='fulltext')
    train_data = lib.file_to_word_ids(vocabulary, train_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=ndf.text_len.max(), max_output=ndf.answers_lstm_len.max())
    valid_data = lib.file_to_word_ids(vocabulary, valid_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=ndf.text_len.max(), max_output=ndf.answers_lstm_len.max())
    test_data = lib.file_to_word_ids(vocabulary, test_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=ndf.text_len.max(), max_output=ndf.answers_lstm_len.max())
    vocabulary_size = len(vocabulary)
    reversed_dictionary = dict(zip(vocabulary.values(), vocabulary.keys()))

    print("First five train ... ", train_data[:5])
    print("Vocabulary   ", vocabulary)
    print(" Size of vocab .  ", vocabulary_size)
    train_data_lst=train_data.tolist()
    print(" ".join([reversed_dictionary[x] for x in train_data_lst[10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size



In [688]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size = load_data()


First five train ...  [[    0     0     0 ...  6438     1 19571]
 [    0     0     0 ... 33090     1 27945]
 [    0     0     0 ... 20644     1 19188]
 [    0     0     0 ... 22106     1 10998]
 [    0     0     0 ... 34450     1 21073]]
Vocabulary    {'': 1, 'leverage': 2, 'spectacular': 3, 'used': 4, '1788': 5, 'kirat': 6, 'kerch': 7, 'interbreed': 8, 'demarcate': 9, 'vimercati': 10, 'scholarly': 11, 'ikea': 12, 'conspiracy': 13, 'solange': 14, 'narrator': 15, 'hund': 16, 'ann': 17, 'riefenstahl': 18, '414': 19, 'dr': 20, 'cinematic': 21, '450': 22, 'mindfulness': 23, 'goetz': 24, 'imɒnˈtænə': 25, 'gland': 26, 'fling': 27, 'buyback': 28, 'tripartite': 29, 'بَنُو': 30, '588': 31, 'gametom': 32, 'acevedo': 33, '11783': 34, 'milwaukee': 35, 'symbolic': 36, 'ecclesia': 37, 'inflate': 38, 'beginning': 39, 'northerncentral': 40, 'leats': 41, 'cinderella': 42, 'shramanic': 43, 'efive': 44, 'snare': 45, 'thirteenth': 46, 'contextembedded': 47, 'schooner': 48, 'enormity': 49, 'fine': 50, 'dip

In [689]:
ndf.text_len.max()

315

In [728]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step
        self.vocabulary_size = len(vocabulary)
        
        
    #     num_steps = nr of words that will be fed into the input layer; i.e. the nr of words the model will use to predict the next word
    #     skip_steps = how far to move the window after the prediction is made

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps)) # input nn nodes, same dimensions as batchsize x nr of words in one "window"
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary_size)) # output nodes
        j=0
        while True:
            for i in range(self.batch_size):
#                 if self.current_idx + self.num_steps >= len(self.data): # once the ticker goes over the length of data, reset it
#                     self.current_idx = 0
#                 x[i,:] = self.data[self.current_idx:self.current_idx + self.num_steps]
                x[i,:] = self.data[j][:-2]
                # input is the segment of data from current index to current index + step-increment
#                 temp_y = self.data[self.current_idx+1 : self.num_steps + self.current_idx + 1]
                temp_y = self.data[j][-1]
                # converts all the output y into a 1-hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary_size)
#                 self.current_idx += self.skip_step
                j+=1
                
            yield x, y
        
            
        # x dimensions: nr of samples we specify in the batch first, number of w to base predictions on 
        # y has 3 dims: batchsize, nr time steps and also size of vocabulary


In [729]:
ndf.text_len.max()

315

In [730]:
num_steps = ndf.text_len.max()
batch_size = 10


In [731]:
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps+1)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps+1)

In [732]:
train_data_generator.generate()

<generator object KerasBatchGenerator.generate at 0x1c497ab360>

In [733]:
hidden_size = 300
use_dropout=True
num_epochs=20


In [734]:
vocabulary['']

1

In [735]:
from keras.activations import sigmoid
from keras.losses import cosine_proximity

In [736]:
# model = Sequential([
#     layers.Embedding(input_dim=vocabulary_size, output_dim=hidden_size,input_length=num_steps, mask_zero=True),
#     layers.LSTM(hidden_size, return_sequences=True),
#     layers.LSTM(hidden_size, return_sequences=True),
#     layers.LSTM(hidden_size, dropout=0.5, return_sequences=True),
#     layers.TimeDistributed(Dense(vocabulary_size)),
#     layers.Activation(sigmoid)
# ])

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/tensor_util.py:560: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [742]:
model = Sequential()
model.add(Embedding(vocabulary_size, hidden_size, input_length=num_steps, mask_zero=False))
model.add((Masking(mask_value=0)))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.25))
model.add(TimeDistributed(Dense(vocabulary_size)))
model.add(Activation(sigmoid))

In [743]:

optimizer=Adam()
model.compile(loss='categorical_hinge', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_36 (Embedding)     (None, 315, 300)          11077800  
_________________________________________________________________
masking_15 (Masking)         (None, 315, 300)          0         
_________________________________________________________________
lstm_71 (LSTM)               (None, 315, 300)          721200    
_________________________________________________________________
lstm_72 (LSTM)               (None, 315, 300)          721200    
_________________________________________________________________
dropout_26 (Dropout)         (None, 315, 300)          0         
_________________________________________________________________
time_distributed_32 (TimeDis (None, 315, 36926)        11114726  
_________________________________________________________________
activation_32 (Activation)   (None, 315, 36926)        0         
Total para

In [712]:
len(vocabulary)

36926

In [701]:
checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)
# yet to implement this

In [703]:
len(valid_data)//(batch_size)

50

In [740]:
len(train_data)//(batch_size*20)

20

In [744]:
model.fit_generator(train_data_generator.generate(), steps_per_epoch = len(train_data)//(batch_size*20), 
                    epochs=num_epochs,)
#                     validation_data=valid_data_generator.generate(),
#                     validation_steps=len(valid_data)//(batch_size))

Epoch 1/20
11/20 [===============>..............] - ETA: 2:19 - loss: 1.0011 - categorical_accuracy: 0.0021

KeyboardInterrupt: 

In [160]:
model.layers

In [163]:
vocabulary_size

3414

In [164]:
model.input

<tf.Tensor 'embedding_6_input:0' shape=(?, 206) dtype=float32>

In [162]:
model.output

<tf.Tensor 'activation_6/truediv:0' shape=(?, 206, 3414) dtype=float32>

In [201]:
dummy_iters = 40
example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=num_steps+1)
print("Training data:")
for i in range(dummy_iters):
    dummy = next(example_training_generator.generate())
num_predict = 110
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps-1, :])
    
    true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
print(true_print_out)
print(pred_print_out)
print(prediction.shape)

Training data:


IndexError: list index out of range

In [ ]:

if args.run_opt ==1:
    model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,)
    #                         validation_data=valid_data_generator.generate(),
    #                         validation_steps=len(valid_data)//(batch_size))
    model.save(data_path + "/final_model.hdf5")
elif args.run_opt ==2:
    model = load_model(data_path + "/mo")
